In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

## detecting and cropping faces

In [3]:
done = False
done = True

from gaze import init, Data
args = init(batch_size=8, no_cuda=True, verbose=1)
d = Data(args)

Found 1358 sample images;  1087  to train 271 to test


In [4]:
import os
import glob
datapath_faces = 'dataset_faces'
datapath = 'dataset'

if True:
    n_show = 10
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png')[-n_show:]:
            print(filename)

dataset/blink/2018-09-12_Laurent_035.png
dataset/blink/2018-09-12_Laurent_223.png
dataset/blink/2018-09-12_Laurent_237.png
dataset/blink/2018-09-12_Laurent_222.png
dataset/blink/2018-09-14_Laurent_121.png
dataset/blink/2018-09-04_Laurent_180.png
dataset/blink/2018-10-03_Laurent_116.png
dataset/blink/2018-09-12_Laurent_020.png
dataset/blink/2018-09-05_Laurent_085.png
dataset/blink/2018-09-14_Laurent_069.png
dataset/center/2018-09-05_Laurent_091.png
dataset/center/2018-09-04_Laurent_16.png
dataset/center/2018-09-12_Laurent_140.png
dataset/center/2018-09-14_Laurent_243.png
dataset/center/2018-09-12_Laurent_168.png
dataset/center/2018-09-12_Laurent_183.png
dataset/center/2018-09-04_Laurent_221.png
dataset/center/2018-09-04_Laurent_235.png
dataset/center/2018-10-03_Laurent_089.png
dataset/center/2018-10-03_Laurent_076.png
dataset/left/2018-09-12_Laurent_236.png
dataset/left/2018-09-05_Laurent_091.png
dataset/left/2018-09-12_Laurent_008.png
dataset/left/2018-09-05_Laurent_052.png
dataset/lef

In [5]:
import imageio
if not done:
    import time
    from gaze import FaceExtractor
    FE = FaceExtractor()
    timings = []
    for target in d.classes:
        for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
            frame = imageio.imread(filename)
            t0 = time.time()
            face = FE.face_extractor(frame)
            t1 = time.time()
            timings.append(t1-t0)
            if False:
                print("Number of faces detected: {}".format(len(dets)))
                for i, d in enumerate(dets):
                    print("Detection {}: Left: {} Top: {} Right: {} Bottom: {}".format(
                        i, d.left(), d.top(), d.right(), d.bottom()))

            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the image
                ax.imshow(frame)

                # Create a Rectangle patch
                rect = patches.Rectangle((d.bottom(), d.left()), d.right()-d.left(), d.top()-d.bottom(), linewidth=1, edgecolor='r', facecolor='none')

                # Add the patch to the Axes
                ax.add_patch(rect)
                plt.show()    
            if False:
                # Create figure and axes
                fig, ax = plt.subplots(figsize=(15, 8))

                # Display the cropped image
                ax.imshow(face)

                plt.show()                

            filename_face = filename.replace(datapath, datapath_faces)
            imageio.imwrite(filename_face, face) 


In [6]:
if not done:
    timings = np.array(timings) * 1000
    print('timings in ms =', timings.mean(), '+/-', timings.std()) 

In [7]:
if not done:
    fig, ax = plt.subplots(figsize=((8, 5)))
    n, bins, patches = ax.hist(timings, bins=np.linspace(100, 200, 100), alpha=.4)
    ax.vlines(np.median(timings), 0, n.max(), 'g', linestyles='dashed', label='median = %.3f ms' % np.median(timings))
    #ax.vlines(25, 0, n.max(), 'r', linestyles='dashed', label='chance level')
    #ax.vlines(100, 0, n.max(), 'k', label='max')
    ax.set_xlabel('Timings (ms)')
    ax.set_ylabel('Smarts')
    ax.legend(loc='best')
    if False:
        plt.show() 
    else:
        plt.savefig('dlib_timings.png')

## training the model on the cropped faces

In [8]:
from gaze import init
#epochs = 400
#args = init(verbose=0, epochs=epochs)
args = init(verbose=1)
print('Parameters=', args)

path = '_Regard.pt'

Parameters= {'dataset_folder': 'dataset', 'dataset_faces_folder': 'dataset_faces', 'batch_size': 8, 'test_batch_size': 1, 'size_test_set': 0.2, 'epochs': 20, 'do_adam': False, 'lr': 0.025, 'momentum': 0.05, 'no_cuda': False, 'num_processes': 1, 'seed': 42, 'log_interval': 0, 'fullsize': 75, 'crop': 64, 'size': 64, 'mean': 0.6, 'std': 0.3, 'conv1_dim': 9, 'conv1_kernel_size': 18, 'conv2_dim': 36, 'conv2_kernel_size': 14, 'conv1_pdropout': 0.1, 'conv2_pdropout': 0.1, 'stride1': 2, 'stride2': 4, 'N_cv': 20, 'dimension': 30, 'verbose': 1}


In [9]:
from gaze import init
epochs = 400
args = init(verbose=0, epochs=epochs)
path = '_Regard_400.pt'

In [10]:
from gaze import init
epochs = 40
args = init(verbose=0, epochs=epochs)
print('Parameters=', args)
path = '_Regard_dropout.pt'

Parameters= {'dataset_folder': 'dataset', 'dataset_faces_folder': 'dataset_faces', 'batch_size': 8, 'test_batch_size': 1, 'size_test_set': 0.2, 'epochs': 40, 'do_adam': False, 'lr': 0.025, 'momentum': 0.05, 'no_cuda': False, 'num_processes': 1, 'seed': 42, 'log_interval': 0, 'fullsize': 75, 'crop': 64, 'size': 64, 'mean': 0.6, 'std': 0.3, 'conv1_dim': 9, 'conv1_kernel_size': 18, 'conv2_dim': 36, 'conv2_kernel_size': 14, 'conv1_pdropout': 0.1, 'conv2_pdropout': 0.1, 'stride1': 2, 'stride2': 4, 'N_cv': 20, 'dimension': 30, 'verbose': 0}


In [11]:
!ls -l {path}

!rm {path}

-rw-r--r--  1 laurentperrinet  staff  308410 Oct  5 20:27 _Regard_dropout.pt


In [12]:
from gaze import ML
ml = ML(args)
ml.train(path=path)

Training model...


Process Process-17:
Traceback (most recent call last):
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/local/Cellar/python/3.6.5_1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/local/lib/python3.6/site-packages/torch/utils/data/dataset.py", line 103, in __getitem__
    return self.dataset[self.indices[idx]]
  File "/usr/local/lib/python3.6/site-packages/torchvision/datasets/folder.py", line 101, in __getitem__
    sample

KeyboardInterrupt: 

## testing the model

In [ ]:
Accuracy = ml.test()
print('Accuracy={:.1f}%'.format(Accuracy*100))

Showing the one which are wrong:

In [ ]:
import time
from gaze import FaceExtractor
FE = FaceExtractor()
timings = []
for target in d.classes:
    for filename in glob.glob(os.path.join(datapath, target) + '/*.png'):
        frame = imageio.imread(filename)
        img_face = FE.face_extractor(frame)
        pred = ml.classify(img_face, ml.dataset.test_transform)
        pred_label = ml.dataset.dataset.classes[pred.argmax()]
        if not pred_label == target:
            print('For filename', filename, ', Prediction =', pred_label, 'Shown label =', target)        